<img src=https://i.imgur.com/QnKVI6k.jpg>

---

# Introduction

In this notebook you will be guided through the extraction, transformation and loading processes of the dataframes needed for our proyect. 

We begin importing all the necessary libraries for the ETL: 

In [1]:
#Libraries to work on the databases:
from pandas_datareader import wb
import wbgapi as wb
import datetime
import datapackage 

#Libraries to work on the pipeline:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

This block of code keeps the datasets up to date:

In [2]:
today = datetime.date.today()
year = today.year

---

# ETL = EXTRACT

We begin the extraction of different datasets from different sources. We divide them by theme:

* Economy:

In [3]:
economy_worldbank_series = ['NY.GDP.MKTP.CD', 'NY.GDP.MKTP.KD.ZG', 'NE.CON.TOTL.KD.ZG', 'NY.GNP.PCAP.CD', 'NY.GNS.ICTR.ZS', 'FP.CPI.TOTL', 
                            'FI.RES.TOTL.CD', 'BX.KLT.DINV.CD.WD', 'IC.TAX.TOTL.CP.ZS', 'IC.REG.DURS', 'NE.EXP.GNFS.ZS', 'NE.IMP.GNFS.ZS']
                                # We indicate which data from the economy series we want.
eco_complete = wb.data.DataFrame((economy_worldbank_series), labels = True, time=range(2000, year), skipBlanks=True, columns='series').reset_index()
                                # We indicate the range of years we want and various details to have a clean database.
eco_complete.rename(columns={'NY.GDP.MKTP.CD': 'Gross Domestic Product', 
                             'NY.GDP.MKTP.KD.ZG': 'GDP Growth', 
                             'NE.CON.TOTL.KD.ZG': 'Final Consumption Expenditure',
                             'NY.GNP.PCAP.CD': 'GNI Per Capita', 
                             'NY.GNS.ICTR.ZS': 'Gross Savings', 
                             'FP.CPI.TOTL': 'Consumer Price', 
                             'FI.RES.TOTL.CD' : 'Total reserves (gold + US$)', 
                             'BX.KLT.DINV.CD.WD' : 'Foreign direct investment, net inflows (BoP, current US$)', 
                             'IC.TAX.TOTL.CP.ZS' : 'Total tax and contribution rate (PCT of profit)', 
                             'IC.REG.DURS' : 'Time required to start a business (days)', 
                             'NE.EXP.GNFS.ZS' : 'Exports of goods and services (PCT of GDP)', 
                             'NE.IMP.GNFS.ZS' : 'Imports of goods and services (PCT of GDP)'}, inplace=True)
                                # In this stage, we can rename the columns with what those values represent.

* People: 

In [4]:
people_worldbank_series = ['SE.XPD.TOTL.GD.ZS', 'SL.UEM.TOTL.ZS', 'SE.PRM.CMPT.ZS', 'VC.IHR.PSRC.P5', 'VC.IHR.PSRC.P5']
                                # We indicate which data from the people series we want.
peo_complete = wb.data.DataFrame((people_worldbank_series), labels = True, time=range(2000, year), skipBlanks=True, columns='series').reset_index()
                                # We indicate the range of years we want and various details to have a clean database.
peo_complete.rename(columns={'SE.XPD.TOTL.GD.ZS': 'Expenditure Education', 
                             'SL.UEM.TOTL.ZS': 'Unemployment', 
                             'SE.PRM.CMPT.ZS' : 'Primary completion rate, total (PCT of relevant age group)', 
                             'VC.IHR.PSRC.P5' : 'Intentional homicides (per 100,000 people)'}, inplace=True)
                                # In this stage, we can rename the columns with what those values represent.

* Environment:

In [5]:
enviroment_worldbank_series = ['EG.ELC.ACCS.ZS', 'SH.STA.BASS.UR.ZS']
                                # We indicate which data from the environment series we want.
env_complete = wb.data.DataFrame((enviroment_worldbank_series), labels = True, time=range(2000, year), skipBlanks=True, columns='series').reset_index()
                                # We indicate the range of years we want and various details to have a clean database.
env_complete.rename(columns={'EG.ELC.ACCS.ZS': 'Access Elect.', 
                             'SH.STA.BASS.UR.ZS': 'Basic Sanitation'}, inplace=True)
                                # In this stage, we can rename the columns with what those values represent.


#Population Density
data_url = 'https://datahub.io/world-bank/en.pop.dnst/datapackage.json'     # Storing the dataset into a generic variable:
package = datapackage.Package(data_url)                                     # Loading Data Package into storage
resources = package.resources
for resource in resources:
    if resource.tabular:
        env_pop_density = pd.read_csv(resource.descriptor['path'])          # Loading only tabular data

* Poverty: 

In [6]:
#Maternal Mortality Ratio
data_url = 'https://datahub.io/world-bank/sh.sta.mmrt/datapackage.json'     # Storing the dataset into a generic variable
package = datapackage.Package(data_url)                                     # Loading Data Package into storage
resources = package.resources
for resource in resources:
    if resource.tabular:
        pov_maternal_mortality = pd.read_csv(resource.descriptor['path'])   # Loading only tabular data


poverty_worldbank_series = ['SM.POP.REFG.OR', 'SM.POP.REFG']                
                                # We indicate which data from the environment series we want.
pov_complete = wb.data.DataFrame((poverty_worldbank_series), labels = True, time=range(2000, year), skipBlanks=True, columns='series').reset_index()
                                # We indicate the range of years we want and various details to have a clean database.
pov_complete.rename(columns={'SM.POP.REFG.OR': 'Refugee population by country or territory of origin', 
                             'SM.POP.REFG': 'Refugee population by country or territory of asylum'}, inplace=True)
                                # In this stage, we can rename the columns with what those values represent.

* States: 

In [7]:
states_worldbank_series = ['IT.CEL.SETS.P2', 'SP.POP.TOTL', 'GB.XPD.RSDV.GD.ZS', 'SL.TLF.TOTL.IN']
                            # We indicate which data from the environment series we want.
sta_complete = wb.data.DataFrame((states_worldbank_series), labels = True, time=range(2000, year), skipBlanks=True, columns='series').reset_index()
                            # We indicate the range of years we want and various details to have a clean database.
sta_complete.rename(columns={'IT.CEL.SETS.P2': 'Mobile Subs.', 
                             'SP.POP.TOTL': 'Population Total', 
                             'GB.XPD.RSDV.GD.ZS' : 'Research and development expenditure (PCT of GDP)', 
                             'SL.TLF.TOTL.IN': 'Labour force, total'}, inplace=True)
                            # In this stage, we can rename the columns with what those values represent.

#GDP per capita
data_url = 'https://datahub.io/world-bank/ny.gdp.pcap.pp.cd/datapackage.json'   # Storing the dataset into a generic variable
package = datapackage.Package(data_url)                                         # Loading Data Package into storage
resources = package.resources
for resource in resources:
    if resource.tabular:
        sta_gdp_percapita = pd.read_csv(resource.descriptor['path'])            # Loading only tabular data

* Demographic Indicators: 

In [8]:
mig_demo_url = 'https://population.un.org/wpp/Download/Files/1_Indicators%20(Standard)/EXCEL_FILES/1_General/WPP2022_GEN_F01_DEMOGRAPHIC_INDICATORS_REV1.xlsx'
    # Storing the link in a variable to make the code cleaner.
mig_demo  = pd.read_excel(mig_demo_url, skiprows=15 , header=1 , index_col=False)
    # Importing the excel, indicating we want to skip the first 15 rows, keep row 16 as header, and removing the index column. 

---

# Pipelines

In this section the pipelines are defined. Three are designed, one for each source, and a fourth one is designed as finishing touches once the datasets are merged. 

* World Bank: 

In [11]:
# Dropping columns:
class DropColumn(BaseEstimator, TransformerMixin):
    def __init__(self, columns=['time']):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.drop(columns=self.columns)
        return X

# Renaming columns:
class ColumnRenamer(BaseEstimator, TransformerMixin):
    def __init__(self, old_name, new_name):
        self.old_name = old_name
        self.new_name = new_name
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X.rename(columns={self.old_name: self.new_name})
        return X

# Filling null values with country mean: 
class ImputeWithCountryMean(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        cols_to_impute = X.columns[3:]  
            # We select the columns starting from the fourth onwards.
        self.means = X.groupby('Country Name')[cols_to_impute].apply(lambda x: x.fillna(x.mean()))  
            # We calculate the mean of the same country.
        X[cols_to_impute] = X[cols_to_impute].fillna(self.means)  
            # We fill in the null values with the calculated mean.
        return X
    
# Filling remaning null values with world mean:
class ImputeWithWorldMean(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        cols_to_impute = X.columns[3:]  
            # We select the columns starting from the fourth onwards.
        self.means = X[cols_to_impute].apply(lambda x: x.fillna(x.mean()))  
            # We calculate the mean of the whole column.
        X[cols_to_impute] = X[cols_to_impute].fillna(self.means)  
            # We fill in the null values with the calculated mean.
        return X

    
# Changing 'Year' type of data:
class ChangeDataType(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['Year'] = X['Year'].astype(str)
        return X

#Defining the pipeline:
processes_WB = [
    ('drop_columns', DropColumn(columns=['time'])), 
    ('rename_economy_column', ColumnRenamer(old_name='economy', new_name='Country Code')),
    ('rename_time_column', ColumnRenamer(old_name='Time', new_name='Year')),
    ('rename_country_column', ColumnRenamer(old_name='Country', new_name='Country Name')),
    ('fill_null_mean', ImputeWithCountryMean()), 
    ('fill_null_world_mean', ImputeWithWorldMean()),
    ('change_data_type', ChangeDataType())]

pipeline_WB = Pipeline(processes_WB)

* Datahub:

In [12]:
# Dropping rows which year is prior to 2000:
class DropRowsBefore2000(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X[X['Year'] >= 2000]
        return X
    
# Checking if there are countries and years duplicated:
class DropDuplicates(BaseEstimator, TransformerMixin):
    def __init__(self, columns=["Country Code", "Year"]):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X.drop_duplicates(subset=self.columns)
        return X

# Filling null values with country mean: 
class ImputeWithCountryMean(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        cols_to_impute = X.columns[3:]  
            # We select the columns starting from the fourth onwards.
        self.means = X.groupby('Country Name')[cols_to_impute].apply(lambda x: x.fillna(x.mean()))  
            # We calculate the mean of the whole column.
        X[cols_to_impute] = X[cols_to_impute].fillna(self.means)  
            # We fill in the null values with the calculated mean.
        return X
    
# Filling remaning null values with world mean:
class ImputeWithWorldMean(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        cols_to_impute = X.columns[3:]  
            # We select the columns starting from the fourth onwards.
        self.means = X[cols_to_impute].apply(lambda x: x.fillna(x.mean()))  
            # We calculate the mean of the whole column.
        X[cols_to_impute] = X[cols_to_impute].fillna(self.means)  
            # We fill in the null values with the calculated mean.
        return X

# Changing 'Year' type of data:
class ChangeDataType(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['Year'] = X['Year'].astype(str)
        return X

# Organizing the columns:
class ColumnOrganizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X[['Country Code', 'Country Name', 'Year', 'Value']]
        return X

#Defining the pipeline:
processes_DH = [            
    ('drop_rows_before_2000', DropRowsBefore2000()),        
    ('drop_duplicates', DropDuplicates(columns=["Country Code", "Year"])),
    ('fill_null_mean', ImputeWithCountryMean()), 
    ('fill_null_world_mean', ImputeWithWorldMean()),
    ('change_data_type', ChangeDataType()),
    ('organize_columns', ColumnOrganizer())]

pipeline_DH = Pipeline(processes_DH)

* United Nations: 

In [13]:
# Keeping only selected columns and renaming them:
class ColumnSelectorRenamer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X[['Region, subregion, country or area *', 'ISO3 Alpha-code', 'Year', 'Natural Change, Births minus Deaths (thousands)', 'Rate of Natural Change (per 1,000 population)',
               'Population Growth Rate (percentage)', 'Crude Birth Rate (births per 1,000 population)', 'Median Age, as of 1 July (years)', 'Life Expectancy at Birth, both sexes (years)',
               'Net Number of Migrants (thousands)', 'Net Migration Rate (per 1,000 population)','Infant Mortality Rate (infant deaths per 1,000 live births)', 'Infant Deaths, under age 1 (thousands)']]
        X.columns = ['Country Name', 'Country Code'] + list(X.columns[2:])
        return X[['Country Name', 'Country Code', 'Year', 'Natural Change, Births minus Deaths (thousands)', 'Rate of Natural Change (per 1,000 population)',
                  'Population Growth Rate (percentage)', 'Crude Birth Rate (births per 1,000 population)', 'Median Age, as of 1 July (years)', 'Life Expectancy at Birth, both sexes (years)',
                  'Net Number of Migrants (thousands)', 'Net Migration Rate (per 1,000 population)', 'Infant Mortality Rate (infant deaths per 1,000 live births)', 'Infant Deaths, under age 1 (thousands)']]

# Dropping rows which year is prior to 2000:
class DropRowsBefore2000(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X[X['Year'] >= 2000]
        return X

# Changing 'Year' type of data:
class ChangeYearDataType(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['Year'] = X['Year'].astype(int)
        X['Year'] = X['Year'].astype(str)
        X['Year'] = X['Year'].str.replace('.', '')
        return X
    
# Normalizing countries' names:
class RenameCountries(BaseEstimator, TransformerMixin):
    def __init__(self):
       pass

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        self.country_dict = {'Bolivia (Plurinational State of)': 'Bolivia', 'Brunei Darussalam': 'Brunei', 'Congo': 'Congo, Republic of the', 'Democratic Republic of the Congo': 'Congo, Democratic Republic of the',
                             "Côte d'Ivoire": "Cote d'Ivoire", 'Czechia': 'Czech Republic (Czechia)', 'Eswatini': 'Eswatini (formerly Swaziland)', 'Iran (Islamic Republic of)': 'Iran', 
                             'Kosovo (under UNSC res. 1244)': 'Kosovo', "Lao People's Democratic Republic": 'Laos', 'Republic of Moldova': 'Moldova', 'Myanmar': 'Myanmar (formerly Burma)', 
                             'North Macedonia': 'North Macedonia (formerly Macedonia)', 'Russian Federation': 'Russia', "Dem. People's Republic of Korea": 'North Korea', 'Republic of Korea': 'South Korea',
                             'Syrian Arab Republic': 'Syria', 'Taiwan Province of China': 'Taiwan', 'United Republic of Tanzania': 'Tanzania', 'Turks and Caicos Islands': 'Turkey', 'United States of America': 'United States',
                             'Holy See': 'Vatican City (Holy See)', 'Venezuela (Bolivarian Republic of)': 'Venezuela', 'Viet Nam': 'Vietnam'}
        X['Country Name'] = X['Country Name'].replace(self.country_dict)
        return X

# Defining the pipeline:
processes_UN = [    
    ('column_selector_renamer', ColumnSelectorRenamer()),
    ('drop_rows_before_2000', DropRowsBefore2000()),          
    ('change_year_data_type', ChangeYearDataType()), 
    ('change_country_names', RenameCountries())
    ]

pipeline_UN = Pipeline(processes_UN)

* Finishing touches

In [14]:
# Changing 'Year' type of data back to INT:
class ChangeYearDataType(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['Year'] = X['Year'].astype(int)
        return X

# Removing the rows that are not countries:
class RemoveRows(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        country_list = ['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia',
        'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium',
        'Belize', 'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei',
        'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic',
        'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo, Republic of the', 'Congo, Democratic Republic of the',
        'Costa Rica', 'Cote d\'Ivoire', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic (Czechia)', 'Denmark', 'Djibouti',
        'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
        'Eswatini (formerly Swaziland)', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia',
        'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti',
        'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy',
        'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', 'Kosovo', 'Kuwait', 'Kyrgyzstan',
        'Laos', 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg',
        'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Mauritania',
        'Mauritius', 'Mexico', 'Moldova', 'Monaco', 'Mongolia', 'Montenegro', 'Morocco', 'Mozambique',
        'Myanmar (formerly Burma)', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Zealand', 'Nicaragua', 'Niger',
        'Nigeria', 'North Korea', 'North Macedonia (formerly Macedonia)', 'Norway', 'Oman', 'Pakistan', 'Palau',
        'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Qatar', 'Romania',
        'Russia', 'Rwanda', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Vincent and the Grenadines', 'Samoa',
        'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone',
        'Singapore', 'Slovakia', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 'South Korea',
        'South Sudan', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', 'Sweden', 'Switzerland', 'Syria', 'Taiwan',
        'Tajikistan', 'Tanzania', 'Thailand', 'Timor-Leste', 'Togo', 'Tonga', 'Trinidad and Tobago', 'Tunisia',
        'Turkey', 'Turkmenistan', 'Tuvalu', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom',
        'United States', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Vatican City (Holy See)', 'Venezuela',
        'Vietnam', 'Yemen', 'Zambia', 'Zimbabwe']
        mask = X['Country Name'].isin(country_list)
        X = X[mask]
        return X

# Filling null values with country mean: 
class ImputeWithCountryMean(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        cols_to_impute = X.columns[3:]  
            # We select the columns starting from the fourth onwards.
        self.means = X.groupby('Country Name')[cols_to_impute].apply(lambda x: x.fillna(x.mean()))  
            # We calculate the mean of the whole column.
        X[cols_to_impute] = X[cols_to_impute].fillna(self.means)  
            # We fill in the null values with the calculated mean.
        return X
    
# Filling remaning null values with world mean:
class ImputeWithWorldMean(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        cols_to_impute = X.columns[3:]  
            # We select the columns starting from the fourth onwards.
        self.means = X[cols_to_impute].apply(lambda x: x.fillna(x.mean()))  
            # We calculate the mean of the whole column
        X[cols_to_impute] = X[cols_to_impute].fillna(self.means)  
            # We fill in the null values with the calculated mean.
        return X

# Dropping duplicated registries:
class DropDuplicates(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.drop_duplicates()
        return X

# Sorting the dataset by alphabetical and cronological order:
class SortDataset(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X.sort_values(['Country Name', 'Year']).reset_index(drop=True)
        return X
    
processes_FIN = [    
    ('change_year_data_type', ChangeYearDataType()),
    ('remove_unnecessary_rows', RemoveRows()),           
    ('fill_null_mean', ImputeWithCountryMean()), 
    ('fill_null_world_mean', ImputeWithWorldMean()),
    ('drop_duplicates', DropDuplicates()),
    ('sort_dataset', SortDataset())
    ]

pipeline_FIN = Pipeline(processes_FIN)

---

# ETL = TRANSFORM

In [22]:
import warnings
warnings.filterwarnings("ignore", message="Not prepending group keys to the result index of transform-like apply.")

* Applying the pipelines: 

In [26]:
# Economy
eco_complete = pipeline_WB.fit_transform(eco_complete)

# People
peo_complete= pipeline_WB.fit_transform(peo_complete)

# Environment
env_complete = pipeline_WB.fit_transform(env_complete)
env_pop_density = pipeline_DH.fit_transform(env_pop_density)

# Poverty
pov_maternal_mortality = pipeline_DH.fit_transform(pov_maternal_mortality)
pov_complete = pipeline_WB.fit_transform(pov_complete)

# States
sta_complete = pipeline_WB.fit_transform(sta_complete)
sta_gdp_percapita = pipeline_DH.fit_transform(sta_gdp_percapita)

# Migration
mig_demo = pipeline_UN.fit_transform(mig_demo)

C:\Users\mbele\AppData\Local\Temp\ipykernel_11496\1769637852.py:41: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X['Year'] = X['Year'].str.replace('.', '')


* Finishing Transformations: 

In [27]:
'''---------Dropping the Vatican City because its data cannot be changed to float:---------'''

mig_demo = mig_demo.drop(mig_demo[mig_demo['Country Name'] == 'Vatican City (Holy See)'].index)

In [28]:
'''--------------------------Changing the type of data to float:--------------------------'''
column_names = mig_demo.columns

for col in column_names[3:]:
        # We change all the columns except for the first 3.
    mig_demo[col] = mig_demo[col].astype('float64')

In [29]:
'''--------------------------------Renaming column 'Value':--------------------------------'''

env_pop_density = env_pop_density.rename(columns={'Value': 'Population Density'})
pov_maternal_mortality = pov_maternal_mortality.rename(columns={'Value': 'Maternal Mortality'})
sta_gdp_percapita = sta_gdp_percapita.rename(columns={'Value': 'GDP per_capita'})

In [31]:
'''-------------------------Merging all of the datasets together:-------------------------'''

merged = eco_complete.merge(peo_complete, on=['Country Code', 'Country Name', 'Year'], 
                            how='outer').merge(env_complete, on=['Country Code', 'Country Name', 'Year'], 
                            how='outer').merge(env_pop_density, on=['Country Code', 'Country Name', 'Year'],
                            how='outer').merge(pov_maternal_mortality, on=['Country Code', 'Country Name', 'Year'], 
                            how='outer').merge(pov_complete, on=['Country Code', 'Country Name', 'Year'], 
                            how='outer').merge(sta_complete, on=['Country Code', 'Country Name', 'Year'], 
                            how='outer').merge(sta_gdp_percapita, on=['Country Code', 'Country Name', 'Year'], 
                            how='outer').merge(mig_demo, on=['Country Code', 'Country Name', 'Year'], how='outer')

In [32]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8137 entries, 0 to 8136
Data columns (total 40 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Country Code                                                 7059 non-null   object 
 1   Country Name                                                 8137 non-null   object 
 2   Year                                                         8137 non-null   object 
 3   Foreign direct investment, net inflows (BoP, current US$)    5738 non-null   float64
 4   Total reserves (gold + US$)                                  5738 non-null   float64
 5   Consumer Price                                               5738 non-null   float64
 6   Time required to start a business (days)                     5738 non-null   float64
 7   Total tax and contribution rate (PCT of profit)              5738 non-null   f

In [33]:
'''----------Applying pipeline to the final dataset:----------'''

processed_merged = pipeline_FIN.fit_transform(merged)  

C:\Users\mbele\AppData\Local\Temp\ipykernel_11496\2677863533.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[cols_to_impute] = X[cols_to_impute].fillna(self.means)
C:\Users\mbele\AppData\Local\Temp\ipykernel_11496\2677863533.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[cols_to_impute] = X[cols_to_impute].fillna(self.means)


---

## Finishing EDA
    We check if everything is correct before exporting.

In [34]:
processed_merged

,Country Code,Country Name,Year,"Foreign direct investment, net inflows (BoP, current US$)",Total reserves (gold + US$),Consumer Price,Time required to start a business (days),Total tax and contribution rate (PCT of profit),Final Consumption Expenditure,Exports of goods and services (PCT of GDP),...,"Natural Change, Births minus Deaths (thousands)","Rate of Natural Change (per 1,000 population)",Population Growth Rate (percentage),"Crude Birth Rate (births per 1,000 population)","Median Age, as of 1 July (years)","Life Expectancy at Birth, both sexes (years)",Net Number of Migrants (thousands),"Net Migration Rate (per 1,000 population)","Infant Mortality Rate (infant deaths per 1,000 live births)","Infant Deaths, under age 1 (thousands)"
0,AFG,Afghanistan,2000,1.700000e+05,6.891865e+09,112.718863,9.062500,43.753333,3.806002,41.106221,...,753.278,37.568,-1.299,49.664,14.127,55.298,-1007.135,-50.229,90.813,89.811
1,AFG,Afghanistan,2001,6.800000e+05,6.891865e+09,112.718863,9.062500,43.753333,3.806002,41.106221,...,737.451,37.266,2.769,48.979,14.059,55.798,-192.286,-9.717,88.358,85.059
2,AFG,Afghanistan,2002,5.000000e+07,6.891865e+09,112.718863,9.062500,43.753333,3.806002,41.106221,...,751.008,36.921,9.904,48.201,14.025,56.454,1327.074,65.242,85.779,83.614
3,AFG,Afghanistan,2003,5.780000e+07,6.891865e+09,112.718863,9.062500,43.753333,3.806002,41.106221,...,823.031,36.652,5.352,47.350,14.017,57.344,388.632,17.307,82.600,87.396
4,AFG,Afghanistan,2004,1.869000e+08,6.891865e+09,63.523395,9.500000,43.753333,3.806002,41.106221,...,853.793,36.053,2.570,46.330,14.033,57.944,-248.616,-10.498,79.936,87.383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4236,ZWE,Zimbabwe,2017,3.071877e+08,2.926212e+08,105.508414,61.000000,31.600000,3.920652,19.658905,...,358.458,24.250,2.024,32.516,17.304,60.709,-59.918,-4.054,38.724,18.619
4237,ZWE,Zimbabwe,2018,7.178653e+08,8.695109e+07,116.712211,32.000000,31.600000,-0.462873,26.163973,...,363.531,24.102,2.017,32.074,17.478,61.414,-59.918,-3.972,37.354,18.057
4238,ZWE,Zimbabwe,2019,2.495000e+08,1.512405e+08,414.684309,27.000000,31.600000,-10.119249,27.163459,...,361.162,23.475,1.962,31.518,17.666,61.292,-59.918,-3.895,37.195,18.038
4239,ZWE,Zimbabwe,2020,1.503600e+08,3.340502e+07,2725.312815,84.764706,36.040000,-4.540536,25.917014,...,358.842,22.877,2.099,31.009,17.866,61.124,-29.955,-1.910,36.810,17.903


In [35]:
processed_merged.shape

(4241, 40)

In [36]:
processed_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4241 entries, 0 to 4240
Data columns (total 40 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Country Code                                                 4241 non-null   object 
 1   Country Name                                                 4241 non-null   object 
 2   Year                                                         4241 non-null   int32  
 3   Foreign direct investment, net inflows (BoP, current US$)    4241 non-null   float64
 4   Total reserves (gold + US$)                                  4241 non-null   float64
 5   Consumer Price                                               4241 non-null   float64
 6   Time required to start a business (days)                     4241 non-null   float64
 7   Total tax and contribution rate (PCT of profit)              4241 non-null   f

In [37]:
processed_merged.describe()

,Year,"Foreign direct investment, net inflows (BoP, current US$)",Total reserves (gold + US$),Consumer Price,Time required to start a business (days),Total tax and contribution rate (PCT of profit),Final Consumption Expenditure,Exports of goods and services (PCT of GDP),Imports of goods and services (PCT of GDP),Gross Domestic Product,...,"Natural Change, Births minus Deaths (thousands)","Rate of Natural Change (per 1,000 population)",Population Growth Rate (percentage),"Crude Birth Rate (births per 1,000 population)","Median Age, as of 1 July (years)","Life Expectancy at Birth, both sexes (years)",Net Number of Migrants (thousands),"Net Migration Rate (per 1,000 population)","Infant Mortality Rate (infant deaths per 1,000 live births)","Infant Deaths, under age 1 (thousands)"
count,4241.000000,4.241000e+03,4.241000e+03,4241.000000,4241.000000,4241.000000,4241.000000,4241.000000,4241.000000,4.241000e+03,...,4241.000000,4241.000000,4241.000000,4241.000000,4241.000000,4241.000000,4241.000000,4241.000000,4241.000000,4241.000000
mean,2010.489979,9.597135e+09,5.090979e+10,148.519058,32.036347,43.234992,3.870699,42.302947,48.011349,2.600151e+11,...,418.833062,13.771659,1.392919,22.166800,26.924557,69.878752,-6.128968,0.236038,27.483469,27.008496
std,6.341869,3.085104e+10,2.115124e+11,648.956514,41.692710,24.380357,7.770523,27.606142,24.962035,8.530424e+11,...,1478.916494,11.008697,1.629787,11.093703,8.810450,8.965270,159.282768,13.319407,25.976926,108.989782
min,2000.000000,-3.443754e+11,2.677074e+05,2.909082,0.500000,8.400000,-40.868215,0.459601,0.297274,1.396504e+07,...,-1061.083000,-13.902000,-11.988000,5.579000,13.777000,41.957000,-2290.411000,-130.308000,0.844000,0.001000
25%,2005.000000,1.393728e+08,1.149327e+09,88.562192,13.176471,32.400000,2.162391,26.025629,31.494170,6.395712e+09,...,4.061000,4.095000,0.419000,12.262000,18.888000,63.890000,-20.921000,-3.123000,7.204000,0.259000
50%,2010.000000,1.118400e+09,1.129439e+10,107.726338,27.300000,41.900000,3.806002,41.106221,45.660348,3.388681e+10,...,55.164000,13.384000,1.270000,19.813000,25.402000,71.734000,-1.399000,-0.461000,17.242000,2.185000
75%,2016.000000,9.597135e+09,5.090979e+10,134.002037,35.000000,46.800000,4.908315,47.562655,54.821251,2.600151e+11,...,333.327000,23.423000,2.308000,30.432000,34.812000,76.622000,10.578000,2.095000,41.364000,16.267000
max,2021.000000,7.338265e+11,3.900039e+12,22570.711031,697.000000,280.200000,393.570492,228.993771,208.332944,1.773406e+13,...,19607.906000,38.112000,21.262000,52.659000,54.524000,86.542000,1327.074000,222.940000,139.216000,1936.554000


In [38]:
# List of unique years in the DF:
processed_merged['Year'].unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021])

In [39]:
# List of unique countries' names:
processed_merged['Country Name'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cabo Verde',
       'Cambodia', 'Cameroon', 'Canada', 'Central African Republic',
       'Chad', 'Chile', 'China', 'Colombia', 'Comoros',
       'Congo, Democratic Republic of the', 'Congo, Republic of the',
       'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus',
       'Czech Republic (Czechia)', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini (formerly Swaziland)', 'Ethiopia', 'Fiji', 'Finland',
       'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana',
       'Gr

In [40]:
# Amount of countries in the final dataset:
processed_merged['Country Name'].nunique()

192

---

# ETL = LOADING

In [52]:
processed_merged.to_csv('Merged_Dataset_v03.csv', index = False)

---

---